In [1]:
print("hello world")



hello world


In [2]:
import pandas as pd

train1 = pd.read_csv("train.csv")
test1 = pd.read_csv("test.csv")
print(train1.shape)
print(test1.shape)

(10312, 3)
(2210, 2)


In [3]:
train1.head(2)

,id,dialogue,summary
0,13818399,Max: How's your job hunt going?\r\nDorothy: Uh...,B has been looking for a job and sending a hun...
1,13865364,Jim: We should meet at the cathedral church\nT...,"A, Tim and Margaret will meet at the cathedral..."


In [4]:
print("DIALOGUE:\n")
print(train1.loc[0, "dialogue"])

print("\nSUMMARY:\n")
print(train1.loc[0, "summary"])

DIALOGUE:

Max: How's your job hunt going?
Dorothy: Uhh, I send a hundred CVs a day
Max: a hundred?!
Dorothy: yeah I mean I don't even care if I fit the profile, I see the ad, I send my CV :D
Max: that's one way of looking at it xD
Dorothy: If even 10% of the companies call me back it's still gonna look good
Max: True
Max: Good luck!!
Dorothy: Thanks :)

SUMMARY:

B has been looking for a job and sending a hundred CVs a day even when she doesn't meet the criteria. She hopes for a 10% response rate.


In [5]:
%pip install transformers sentencepiece torch


Note: you may need to restart the kernel to use updated packages.


In [6]:
from huggingface_hub import login
login()



In [7]:
%pip install huggingface_hub

Note: you may need to restart the kernel to use updated packages.


In [8]:
from transformers import pipeline

summarizer = pipeline(
    "summarization",
    model="facebook/bart-large-cnn"
)


Device set to use cpu


In [9]:
import pandas as pd

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

print(train.shape)
print(test.shape)


(10312, 3)
(2210, 2)


In [10]:
train.head(2)


,id,dialogue,summary
0,13818399,Max: How's your job hunt going?\r\nDorothy: Uh...,B has been looking for a job and sending a hun...
1,13865364,Jim: We should meet at the cathedral church\nT...,"A, Tim and Margaret will meet at the cathedral..."


In [11]:
dialogue = train.loc[0, "dialogue"]

print("DIALOGUE:\n")
print(dialogue)

print("\nMODEL SUMMARY:\n")
summary = summarizer(dialogue, max_length=60, min_length=15, do_sample=False)
print(summary[0]["summary_text"])


DIALOGUE:

Max: How's your job hunt going?
Dorothy: Uhh, I send a hundred CVs a day
Max: a hundred?!
Dorothy: yeah I mean I don't even care if I fit the profile, I see the ad, I send my CV :D
Max: that's one way of looking at it xD
Dorothy: If even 10% of the companies call me back it's still gonna look good
Max: True
Max: Good luck!!
Dorothy: Thanks :)

MODEL SUMMARY:

"I send a hundred CVs a day," says Dorothy. "I don't even care if I fit the profile," she says. "If even 10% of the companies call me back it's still gonna look good," she adds.


In [12]:
print(train.loc[0, "dialogue"])
print("\nSUMMARY:")
print(train.loc[0, "summary"])

Max: How's your job hunt going?
Dorothy: Uhh, I send a hundred CVs a day
Max: a hundred?!
Dorothy: yeah I mean I don't even care if I fit the profile, I see the ad, I send my CV :D
Max: that's one way of looking at it xD
Dorothy: If even 10% of the companies call me back it's still gonna look good
Max: True
Max: Good luck!!
Dorothy: Thanks :)

SUMMARY:
B has been looking for a job and sending a hundred CVs a day even when she doesn't meet the criteria. She hopes for a 10% response rate.


In [13]:
from transformers import pipeline

summarizer = pipeline(
    "summarization",
    model="sshleifer/distilbart-cnn-12-6"
)


Device set to use cpu


In [14]:
 def generate_summary(text):
    input_length = len(text.split())

    max_len = max(12, int(0.35 * input_length))
    min_len = max(8, int(0.15 * input_length))

    return summarizer(
        text,
        max_length=max_len,
        min_length=min_len,
        do_sample=False
    )[0]["summary_text"]


In [15]:
import re

def clean_dialogue(dialogue):
    return re.sub(r"\b[A-Za-z]+:\s*", "", dialogue)


dialogue = clean_dialogue(train.loc[0, "dialogue"])
print(generate_summary(dialogue))


 "If even 10% of the companies call me back it's still gonna look good," says


In [16]:
def clean_abstractive_output(summary):
    phrases = [
        " says ",
        " she says",
        " he says",
        " she adds",
        " he adds"
    ]
    for p in phrases:
        summary = summary.replace(p, " ")
    return summary.strip().capitalize()


In [17]:
raw_summary = generate_summary(clean_dialogue(dialogue))
final_summary = clean_abstractive_output(raw_summary)

print(final_summary)


"if even 10% of the companies call me back it's still gonna look good," says


In [18]:
import re
from collections import Counter
import string

def split_sentences_simple(text):
    # split by line breaks or sentence punctuation
    sentences = re.split(r"[.\n!?]+", text)
    return [s.strip() for s in sentences if len(s.strip()) > 0]


def extractive_summary(dialogue, top_n=1):
    sentences = split_sentences_simple(dialogue)

    words = [
        w.lower().strip(string.punctuation)
        for w in dialogue.split()
        if w.isalpha()
    ]

    freq = Counter(words)

    sentence_scores = {}
    for sent in sentences:
        score = sum(
            freq.get(w.lower().strip(string.punctuation), 0)
            for w in sent.split()
        )
        sentence_scores[sent] = score

    ranked = sorted(sentence_scores, key=sentence_scores.get, reverse=True)
    return " ".join(ranked[:top_n])


In [19]:
print("DIALOGUE:\n", dialogue)
print("\nHUMAN SUMMARY:\n", train.loc[0, "summary"])
print("\nEXTRACTIVE SUMMARY:\n", extractive_summary(dialogue))
print("\nABSTRACTIVE SUMMARY (POST-PROCESSED):\n", final_summary)


DIALOGUE:
 How's your job hunt going?
Uhh, I send a hundred CVs a day
a hundred?!
yeah I mean I don't even care if I fit the profile, I see the ad, I send my CV :D
that's one way of looking at it xD
If even 10% of the companies call me back it's still gonna look good
True
Good luck!!
Thanks :)

HUMAN SUMMARY:
 B has been looking for a job and sending a hundred CVs a day even when she doesn't meet the criteria. She hopes for a 10% response rate.

EXTRACTIVE SUMMARY:
 yeah I mean I don't even care if I fit the profile, I see the ad, I send my CV :D

ABSTRACTIVE SUMMARY (POST-PROCESSED):
 "if even 10% of the companies call me back it's still gonna look good," says


In [20]:
print("DIALOGUE:\n", dialogue)
print("\nHUMAN SUMMARY:\n", train.loc[0, "summary"])
print("\nEXTRACTIVE SUMMARY:\n", extractive_summary(dialogue, top_n=2))
print("\nABSTRACTIVE SUMMARY (POST-PROCESSED):\n", final_summary)


DIALOGUE:
 How's your job hunt going?
Uhh, I send a hundred CVs a day
a hundred?!
yeah I mean I don't even care if I fit the profile, I see the ad, I send my CV :D
that's one way of looking at it xD
If even 10% of the companies call me back it's still gonna look good
True
Good luck!!
Thanks :)

HUMAN SUMMARY:
 B has been looking for a job and sending a hundred CVs a day even when she doesn't meet the criteria. She hopes for a 10% response rate.

EXTRACTIVE SUMMARY:
 yeah I mean I don't even care if I fit the profile, I see the ad, I send my CV :D If even 10% of the companies call me back it's still gonna look good

ABSTRACTIVE SUMMARY (POST-PROCESSED):
 "if even 10% of the companies call me back it's still gonna look good," says


In [21]:
print("\nEXTRACTIVE SUMMARY:\n", extractive_summary(dialogue, top_n=2))



EXTRACTIVE SUMMARY:
 yeah I mean I don't even care if I fit the profile, I see the ad, I send my CV :D If even 10% of the companies call me back it's still gonna look good


In [22]:
dialogue = train.loc[0, "dialogue"]

print("DIALOGUE")
print("-" * 50)
print(dialogue)

print("\nHUMAN SUMMARY")
print("-" * 50)
print(train.loc[0, "summary"])

print("\nEXTRACTIVE SUMMARY")
print("-" * 50)
print(extractive_summary(dialogue, top_n=2))

print("\nABSTRACTIVE SUMMARY")
print("-" * 50)
print(final_summary)


DIALOGUE
--------------------------------------------------
Max: How's your job hunt going?
Dorothy: Uhh, I send a hundred CVs a day
Max: a hundred?!
Dorothy: yeah I mean I don't even care if I fit the profile, I see the ad, I send my CV :D
Max: that's one way of looking at it xD
Dorothy: If even 10% of the companies call me back it's still gonna look good
Max: True
Max: Good luck!!
Dorothy: Thanks :)

HUMAN SUMMARY
--------------------------------------------------
B has been looking for a job and sending a hundred CVs a day even when she doesn't meet the criteria. She hopes for a 10% response rate.

EXTRACTIVE SUMMARY
--------------------------------------------------
Dorothy: yeah I mean I don't even care if I fit the profile, I see the ad, I send my CV :D Dorothy: If even 10% of the companies call me back it's still gonna look good

ABSTRACTIVE SUMMARY
--------------------------------------------------
"if even 10% of the companies call me back it's still gonna look good," says
